<a href="https://colab.research.google.com/github/saqib-sarwar/sts/blob/main/sts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# Upload locally downloaded file from http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
uploaded = files.upload()

Saving Stsbenchmark.tar.gz to Stsbenchmark.tar.gz


In [ ]:
pip install -U sentence-transformers

In [3]:
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import gzip

import pandas as pd
import numpy as np

In [4]:
import tarfile

#Open tar file in read mode and Extract the files into the /tmp folder
file = tarfile.open('/content/Stsbenchmark.tar.gz', 'r') 
file.extractall('/content/temp') 
file.close()

In [5]:
batch_size = 32
epochs = 2
model_save_path = 'content/temp/model_' + datetime.now().strftime("%H:%M:%S")

train_path = "/content/temp/stsbenchmark/sts-train.csv"
dev_path = "/content/temp/stsbenchmark/sts-dev.csv"
test_path = "/content/temp/stsbenchmark/sts-test.csv"

In [6]:
# View contents of file
sts_train = pd.read_csv(train_path, sep="\t", header=None, on_bad_lines="skip")
sts_dev = pd.read_csv(dev_path, sep="\t", header=None, on_bad_lines="skip")
sts_test = pd.read_csv(test_path, sep="\t", engine='python', header=None, on_bad_lines="skip")
sts_train.info()
sts_train.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5509 entries, 0 to 5508
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       5509 non-null   object 
 1   1       5509 non-null   object 
 2   2       5509 non-null   object 
 3   3       5509 non-null   int64  
 4   4       5509 non-null   float64
 5   5       5509 non-null   object 
 6   6       5506 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 301.4+ KB


,0,1,2,3,4,5,6
0,main-captions,MSRvid,2012test,1,5.00,A plane is taking off.,An air plane is taking off.
1,main-captions,MSRvid,2012test,4,3.80,A man is playing a large flute.,A man is playing a flute.
2,main-captions,MSRvid,2012test,5,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,main-captions,MSRvid,2012test,6,2.60,Three men are playing chess.,Two men are playing chess.
4,main-captions,MSRvid,2012test,9,4.25,A man is playing the cello.,A man seated is playing the cello.


In [7]:
# Select a model from SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [8]:

# Prepare data for data_loaders
train_data = []
dev_data = []
test_data = []

# Normalize Scores in range 0-1
train_score = [(x/5.0) for x in sts_train[4]]
dev_score = [(x/5.0) for x in sts_dev[4]]
test_score = [(x/5.0) for x in sts_test[4]]

train_data = [InputExample(texts=[s1, s2], label=t_score) for s1,s2,t_score in zip(sts_train[5], sts_train[6], train_score)]
#For getting symmetric scores, i.e. SentenceTransformer(S1,S2) = SentenceTransformer(S2,S1), we pass both combinations to the train set
train_data = train_data +  [InputExample(texts=[s2, s1], label=t_score) for s1,s2,t_score in zip(sts_train[5], sts_train[6], train_score)]

dev_data = [InputExample(texts=[s1, s2], label=t_score) for s1,s2,t_score in zip(sts_dev[5], sts_dev[6], dev_score)]
test_data = [InputExample(texts=[s1, s2], label=t_score) for s1,s2,t_score in zip(sts_test[5], sts_test[6], test_score)]



In [9]:
# define train data_loader
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)

# Define the loss function
train_loss = losses.CosineSimilarityLoss(model=model)

# Add an evaluator, which evaluates the performance during training
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_data, name='sts-dev')

In [10]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=epochs,
          evaluator=evaluator,
          warmup_steps=100,
          output_path=model_save_path ) 

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/345 [00:00<?, ?it/s]

Iteration:   0%|          | 0/345 [00:00<?, ?it/s]

In [11]:
# Load the saved model
model = SentenceTransformer(model_save_path)

In [12]:
# Evaluate on test data
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_data, name='sts-test')
evaluator(model)

0.8920520980042438